<a href="https://colab.research.google.com/github/p1306/Chatbot-for-VIT-Hostel-Resource-Management/blob/main/Chatbot_for_hostel_resources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
# Load datasets
category_files = {
    "mattress": "/content/drive/MyDrive/capston hostel/Mattress.csv",
    "books": "/content/drive/MyDrive/capston hostel/books.csv",
    "electronics": "/content/drive/MyDrive/capston hostel/electronics.csv",
    "toiletries": "/content/drive/MyDrive/capston hostel/toiletries.csv",
    "clothes": "/content/drive/MyDrive/capston hostel/clothes.csv"
}

dataframes = {cat: pd.read_csv(file) for cat, file in category_files.items()}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train ML models for each category
models = {}

for category, df in dataframes.items():
    # Ensure 'Recommendation' column exists
    if 'recommendation' not in df.columns:
        print(f"Skipping {category}: No 'Recommendation' column found.")
        continue

    # Remove ID column if present
    df = df.drop(columns=["id"], errors="ignore")

    # Encode categorical features
    df_encoded = df.copy()
    encoders = {}

    for col in df.columns:
        if df[col].dtype == object and col != "recommendation":
            encoders[col] = LabelEncoder()
            df_encoded[col] = encoders[col].fit_transform(df[col])

    # Encode target variable
    target_le = LabelEncoder()
    df_encoded["recommendation"] = target_le.fit_transform(df["recommendation"])

    # Split data
    X = df_encoded.drop(columns=["recommendation"])
    y = df_encoded["recommendation"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Store trained model & encoders
    models[category] = {"model": model, "target_encoder": target_le, "feature_encoders": encoders}

print("✅ Models trained successfully!")

✅ Models trained successfully!


In [ ]:
# Store user interactions
user_data = []

In [ ]:
#chatbot
def chatbot():
    print("\n🔹 Welcome to the Recycle System Chatbot 🔹")

    # Select category
    print(f"\nAvailable categories: {list(category_files.keys())}")
    category = input("👉 Please select a category: ").strip().lower()

    if category not in dataframes:
        print("❌ Invalid category! Please restart.")
        return

    # Ask for item name
    item_name = input("🔹 Enter the name of the item: ").strip()

    # Retrieve relevant conditions (excluding ID and Recommendation)
    df = dataframes[category].drop(columns=["id", "recommendation"], errors="ignore")

    user_inputs = {"Item Name": item_name, "Category": category}

    for col in df.columns:
        if df[col].dtype == object:
            # If it's a categorical variable (not YES/NO)
            unique_values = dataframes[category][col].dropna().unique()
            print(f"👉 Choose a value for {col} {list(unique_values)}:")
            value = input().strip()

            while value not in unique_values:
                print(f"❌ Invalid input! Choose one from {list(unique_values)}:")
                value = input().strip()

            user_inputs[col] = value

        elif df[col].nunique() == 2:
            # If it's a binary YES/NO input
            value = input(f"👉 Enter YES or NO for {col}: ").strip().upper()
            while value not in ["YES", "NO"]:
                print("❌ Invalid input! Please enter YES or NO.")
                value = input(f"👉 Enter YES or NO for {col}: ").strip().upper()
            user_inputs[col] = 1 if value == "YES" else 0

        else:
            # If it's a numerical input
            if col in ["overall_condition", "thickness"]:  # Ensure range is 1-5
              value = input(f"👉 Enter a number for {col} (1-5): ").strip()
              while not value.isdigit() or not (1 <= int(value) <= 5):
                print("❌ Invalid input! Please enter a number between 1 and 5.")
                value = input(f"👉 Enter a number for {col} (1-5): ").strip()
              user_inputs[col] = int(value)
            else:
              value = input(f"👉 Enter a number for {col} in months: ").strip()
              user_inputs[col] = int(value)

    # Convert inputs for prediction
    input_df = pd.DataFrame([user_inputs])
    input_df.drop(columns=["Item Name", "Category", "Recommendation"], errors="ignore", inplace=True)

    # Encode categorical features using stored encoders
    model_info = models[category]
    encoders = model_info["feature_encoders"]

    for col, encoder in encoders.items():
        input_df[col] = encoder.transform([user_inputs[col]])

    # Predict recommendation
    prediction = model_info["model"].predict(input_df)
    recommendation = model_info["target_encoder"].inverse_transform(prediction)[0]

    print(f"\n✅ Recommended action for '{item_name}': {recommendation}")

    # Save user data
    user_inputs["Recommendation"] = recommendation
    user_data.append(user_inputs)

    # Save to CSV
    pd.DataFrame(user_data).to_csv("user_responses.csv", index=False)
    # print("\n✅ Data recorded successfully!")

In [ ]:
import spacy
import csv
import os

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Complaint categories with associated keywords
complaint_categories = {
    "electronics": ["fan", "switch", "light", "TV", "computer", "charger", "power"],
    "carpentry": ["door", "window", "furniture", "bed", "chair", "table", "lock", "repair"],
    "AC": ["AC", "air conditioner", "cooling", "temperature"],
    "housekeeping": ["dirty", "cleaning", "dust", "washroom", "hygiene"],
    "plumbing": ["tap", "pipe", "water", "leakage", "flush", "drainage"],
    "mess": ["food", "canteen", "meal", "utensils", "dining hall"]
}

# Function to categorize complaint
def categorize_complaint(complaint_text):
    doc = nlp(complaint_text.lower())  # Process text

    # Check keywords against complaint categories
    for token in doc:
        for category, keywords in complaint_categories.items():
            if token.text in keywords:
                return category  # Return matched category

    return "general"  # Default category if no match is found

# Function to save complaint to CSV
def save_complaint(issue, category, filename="/content/drive/MyDrive/capston hostel/complaints.csv"):
    file_exists = os.path.isfile(filename)

    with open(filename, mode="a", newline="") as file:
        writer = csv.writer(file)

        # Write header if file doesn't exist
        if not file_exists:
            writer.writerow(["Issue", "Category"])

        # Write the complaint entry
        writer.writerow([issue, category])

    print("📂 Complaint logged successfully!")

# Chatbot flow
def chatbot_flow():
    user_choice = input("👉 Do you want to register a complaint or report a left-behind item? (Type 'complaint' or 'left-behind'): ").strip().lower()

    if user_choice == "complaint":
        complaint_text = input("🔹 Please describe your complaint: ")
        category = categorize_complaint(complaint_text)

        print(f"✅ Complaint registered under **{category.upper()}** category.")
        print(f"📌 Issue: {complaint_text}")

        # Save complaint to CSV
        save_complaint(complaint_text, category)

    elif user_choice == "left-behind":
        print("Proceeding with left-behind items flow...")
        chatbot()  # Connect to existing flow

    else:
        print("❌ Invalid choice. Please enter 'complaint' or 'left-behind'.")

# Run chatbot
chatbot_flow()

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


👉 Do you want to register a complaint or report a left-behind item? (Type 'complaint' or 'left-behind'): left-behind
Proceeding with left-behind items flow...

🔹 Welcome to the Recycle System Chatbot 🔹

Available categories: ['mattress', 'books', 'electronics', 'toiletries', 'clothes']
👉 Please select a category: toiletries
🔹 Enter the name of the item: bucket
👉 Choose a value for broken ['NO', 'YES']:
YES
👉 Enter a number for usage_duration in months: 16
👉 Choose a value for working ['YES', 'NO']:
YES

✅ Recommended action for 'bucket': Repair
